# Life Expectancy And GDP

## Include external libraries

In [24]:
import numpy as np
from scipy import stats as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Read From CSV

In [13]:
data = pd.read_csv('all_data.csv')
print(data.head())

  Country  Year  Life expectancy at birth (years)           GDP
0   Chile  2000                              77.3  7.786093e+10
1   Chile  2001                              77.3  7.097992e+10
2   Chile  2002                              77.8  6.973681e+10
3   Chile  2003                              77.9  7.564346e+10
4   Chile  2004                              78.0  9.921039e+10


## Understanding Data Imported from CSV

In [14]:
data.info

<bound method DataFrame.info of      Country  Year  Life expectancy at birth (years)           GDP
0      Chile  2000                              77.3  7.786093e+10
1      Chile  2001                              77.3  7.097992e+10
2      Chile  2002                              77.8  6.973681e+10
3      Chile  2003                              77.9  7.564346e+10
4      Chile  2004                              78.0  9.921039e+10
..       ...   ...                               ...           ...
91  Zimbabwe  2011                              54.9  1.209845e+10
92  Zimbabwe  2012                              56.6  1.424249e+10
93  Zimbabwe  2013                              58.0  1.545177e+10
94  Zimbabwe  2014                              59.2  1.589105e+10
95  Zimbabwe  2015                              60.7  1.630467e+10

[96 rows x 4 columns]>

In [9]:
data.dtypes

Country                              object
Year                                  int64
Life expectancy at birth (years)    float64
GDP                                 float64
dtype: object

In [10]:
data.nunique()

Country                              6
Year                                16
Life expectancy at birth (years)    69
GDP                                 96
dtype: int64

In [11]:
data.isna().sum()

Country                             0
Year                                0
Life expectancy at birth (years)    0
GDP                                 0
dtype: int64

## GDP Analysis

In [18]:
min_gdp = np.min(data.GDP)

In [19]:
max_gdp = np.max(data.GDP)

In [20]:
avg_gdp = np.mean(data.GDP)

In [32]:
mode_gdp = st.mode(data.GDP, keepdims='true')

In [33]:
print("GDP numerical analysis >> Min: {0}, Max: {1}, Avg: {2}".format(min_gdp, max_gdp, avg_gdp))
print(mode_gdp)

GDP numerical analysis >> Min: 4415702800.0, Max: 18100000000000.0, Avg: 3880498570768.396
ModeResult(mode=array([4.4157028e+09]), count=array([1]))
